# How to add human-in-the-loop processes to the prebuilt ReAct agent

This tutorial will show how to add human-in-the-loop processes to the prebuilt ReAct agent. Please see this tutorial for how to get started with the prebuilt ReAct agent

You can add a a breakpoint before tools are called by passing interrupt_before=["tools"] to create_react_agent. Note that you need to be using a checkpointer for this to work.


In [1]:
%pip install -U langgraph langchain-openai


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# First we initialize the model we want to use.
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)


# For this tutorial we will use custom tool that returns pre-defined values for weather in two cities (NYC & SF)

from typing import Literal

from langchain_core.tools import tool


@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown city")


tools = [get_weather]

# We need a checkpointer to enable human-in-the-loop patterns
from langgraph.checkpoint import MemorySaver

memory = MemorySaver()

# Define the graph

from langgraph.prebuilt import create_react_agent

graph = create_react_agent(model, tools=tools, interrupt_before=["tools"], checkpointer=memory)

In [3]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [4]:
config = {"configurable": {"thread_id": "42"}}
inputs = {"messages": [("user", "What's the weather in SF?")]}

print_stream(graph.stream(inputs, config, stream_mode="values"))

================================ Human Message =================================

What's the weather in SF?
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_PssWe9MzfZNVPemyvEeNq41z)
 Call ID: call_PssWe9MzfZNVPemyvEeNq41z
  Args:
    city: sf


In [5]:
snapshot = graph.get_state(config)
print("Next step: ", snapshot.next)

Next step:  ('tools',)


In [6]:
print_stream(graph.stream(None, config, stream_mode="values"))

================================= Tool Message =================================
Name: get_weather

It's always sunny in sf
================================== Ai Message ==================================

The weather in San Francisco is currently sunny.
